In [1]:
import pandas as pd
from datasets import Dataset
import torch

/home/filippo/miniconda3/envs/MNLP_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class OldItalianDataset(Dataset):
    
    def __init__(self, path):
        self.path=path
        self.dataset = pd.read_csv(path)  

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        self.idx=idx
        row = self.dataset.iloc[idx]
        return row["Sentence"]
    
    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

dataset = OldItalianDataset("/home/filippo/Documents/Visual Studio Code/Homework2_MNLP/HW2_MNLP_NonModernIt2ModernIt/cleaned_dataset.csv")  
text=[]

for sentence in dataset:
    text.append(sentence)

In [6]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

pipe = pipeline("text-generation", model="Qwen/Qwen2.5-1.5B")
messages = [
    {"role": "user", "content": "Who are you?"},
]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pipe(messages)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B").to(device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cpu


In [ ]:
def translate_with_Qwen(sentence):
    messages = [
        {
            "role": "system",
            "content": "Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo.",
        },
        {
            "role": "user",
            "content": """Esempi:

Testo antico: "I vendimenti de' morti et le presure de' vivi fece la frode d'uno feroce re."
Traduzione moderna: "Fu l'inganno di un re feroce a provocare le vendette dei morti e le catture dei vivi."

Testo antico: "Gli uomini spessamente a stare fermi nella bugia incontra la verità."
Traduzione moderna: "Gli uomini spesso restano fermi nella menzogna, opponendosi alla verità."

Testo antico: \"""" + sentence + "\"\"\nTraduzione moderna:" 
        },
    ]

    # Apply proper chat template
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    # Generate
    output_ids = model.generate(
        input_ids,
        max_new_tokens=128,
        temperature=0.7,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    # Decode only new output tokens
    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    # Clean up trailing examples
    return output_text.strip().split("Testo antico:")[0].strip()


🟡 Original 1:
quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
🟢 Translated:
 Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo.为抓
为抓user
Esempi:

🟡 Original 2:
crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.


KeyboardInterrupt: 

In [19]:
for i in range(10):
    print(f"Original sentence number {i+1}: {text[i]}")
    print("---------------------")
    print(f"Translated sentence number {i+1}: {translate_with_Qwen(text[i])}")
    print("---------------------")

Original sentence number 1: quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
---------------------
Translated sentence number 1: Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo.为抓
为抓user
Esempi:
---------------------
Original sentence number 2: crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
---------------------


KeyboardInterrupt: 